<a href="https://colab.research.google.com/github/skywalker0803r/Ricky/blob/master/ensemble_submit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load

In [0]:
import pandas as pd

#資料夾1
path42 = '/content/drive/My Drive/玉山人工智慧比賽數據/lightgbm_result/random_state_42/submission_lightgbm_Fold_{}.csv'

#資料夾2
path7 = '/content/drive/My Drive/玉山人工智慧比賽數據/lightgbm_result/random_state_7/submission_lightgbm_Fold_{}.csv'

dfs42 = {}
dfs7 = {}

#迴圈讀取後存入字典
for i in range(0,5,1):
  dfs42[i] = pd.read_csv(path42.format(i),index_col=0)

for i in range(0,5,1):
  dfs7[i] = pd.read_csv(path7.format(i),index_col=0)

# merge

In [3]:
final = pd.DataFrame()
final['txkey'] = dfs42[0]['txkey']

for i in range(0,5,1):
  final['pred_'+str(i)] = dfs42[i]['fraud_ind'].values 

for i in range(0,5,1):
  final['pred_'+str(i+5)] = dfs7[i]['fraud_ind'].values

final.head()

,txkey,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9
1521787,592489,0,0,0,0,0,0,0,0,0,0
1521788,592452,0,0,0,0,0,0,0,0,0,0
1521789,590212,0,0,0,0,0,0,0,0,0,0
1521790,590209,0,0,0,0,0,0,0,0,0,0
1521791,592488,0,0,0,0,0,0,0,0,0,0


# count mean

In [4]:
final['fraud_ind'] = final.drop('txkey',axis=1).mean(axis=1)
final = final[['txkey','fraud_ind']]
final.sample(5)

,txkey,fraud_ind
1888611,902304,0.0
1767673,1175557,0.0
1604412,1323494,0.0
1844448,969937,0.0
1918686,635355,0.0


# value_counts

In [5]:
final['fraud_ind'].value_counts()

0.0    415038
1.0      3154
0.2      1094
0.8       719
0.4       718
0.6       616
0.1       112
0.5        60
0.3        60
0.7        52
0.9        42
Name: fraud_ind, dtype: int64

In [0]:
def f(x):
  if x>=0.4:
    return 1
  else:
    return 0
final['fraud_ind'] = final['fraud_ind'].apply(f)

In [7]:
all_list = list(dfs42.values())+list(dfs7.values())
all_list.append(final)
len(all_list)

11

In [8]:
ps={}

for index,data in enumerate(all_list):
  ps['p'+str(index)] = round((data['fraud_ind'].value_counts()[1] / #盜刷樣本數
                len(data['fraud_ind']))*100 ,5) #總數

pd.DataFrame(ps,index=['盜刷比例%'])

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10
盜刷比例%,1.11866,1.09945,1.13147,1.09376,1.18506,1.11866,1.09945,1.13147,1.0857,1.17321,1.27139


# 保存預測結果

In [0]:
path = "/content/drive/My Drive/玉山人工智慧比賽數據/lightgbm_result/10個lgb_model_voting_big_than_04.csv"
final.to_csv(path)

In [10]:
print(final["fraud_ind"].value_counts())

0    416304
1      5361
Name: fraud_ind, dtype: int64
